In [3]:
# Loads agent from file. Useful to avoid training it again.
import torch
from torch import optim
from dqn_model import Net
import numpy as np
from agent import Agent1 as Agent
from experience_buffer import ExperienceBuffer
from config import HYPERPARAMS
from calc_nash_monopoly import actions_dict
params = HYPERPARAMS['full_obs_NB']
from cont_bertrand import ContBertrand
from testing import testing



REPLAY_SIZE = params['replay_size']
#nA = params['nA']
#LEARNING_RATE = params['learning_rate']
LEARNING_RATE = 0.001
nA = 7
MEAN_C = 1
MEAN_Q = 2
A0 = 1
MU = 1/2
FIRMLIST = []

for c in [MEAN_C]:
    for q in [MEAN_Q]:
        q = round(q, 2)
        FIRMLIST.append({'cost': c, 'quality': q})
NASH_ACTIONS = actions_dict(nA, A0, MU, FIRMLIST, FIRMLIST, "nash")
MONOPOLY_ACTIONS = actions_dict(nA, A0, MU, FIRMLIST, FIRMLIST, "monopoly")


eparams = {
        'firmlist': FIRMLIST,
        'a0': A0,
        'mu': MU,
        'nash_actions': NASH_ACTIONS,
        'monopoly_actions': MONOPOLY_ACTIONS,
        'randomness': 0
        }

firm0 = FIRMLIST[0]
firm1 = firm0
PATH = "3499993_0.1checkpoint.pt" # Special to this specification (dont know why)
checkpoint = torch.load(PATH)
model = Net(2,nA,8).to('cpu')
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE) 
model.load_state_dict(checkpoint['agent_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_dict'])
buffer = ExperienceBuffer(REPLAY_SIZE)
env = ContBertrand(firm0, firm1, eparams)
model.eval()
agent = Agent(env, buffer, model, model, optimizer)

In [4]:
testing(FIRMLIST, agent, params, eparams,'random_test.csv',0)